# Hand Gesture Classification
### by Adrian Abraham

## 1 - Package installation
- **tensorflow**: for model creation and predictions
- **pandas**: for csv_reading
- **numpy**: for data storage and manipulation
- **mediapipe**: contains pre-made hand detection module for data collection
- **open-cv**: for live feed
- **ast**: to convert string literal lists into actual lists

In [1]:
!pip install tensorflow

In [2]:
!pip install pandas

In [3]:
!pip install mediapipe

In [4]:
!pip install opencv-python

## 2 - Importing packages

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ast
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.losses import BinaryCrossentropy

### Quick check that tensorflow is working

In [6]:
print(tf.__version__)

2.16.2


## 3 - Importing data

In [35]:
# Positive and negative data are stored in csv format.
# We can use read_csv() to get the contents of each file
df1 = pd.read_csv('peace.csv')
df0 = pd.read_csv('not_peace.csv')

In [36]:
df1.info

<bound method DataFrame.info of                                                  coords
0     [(0.6682692307692307, 1.0), (0.384615384615384...
1     [(0.6634615384615384, 1.0), (0.379807692307692...
2     [(0.6650717703349283, 1.0), (0.382775119617224...
3     [(0.6523809523809524, 1.0), (0.376190476190476...
4     [(0.6555023923444976, 1.0), (0.377990430622009...
...                                                 ...
1979  [(0.9063545150501672, 1.0), (0.658862876254180...
1980  [(0.9060402684563759, 1.0), (0.657718120805369...
1981  [(0.9093959731543624, 1.0), (0.664429530201342...
1982  [(0.9078947368421053, 1.0), (0.661184210526315...
1983  [(0.9174917491749175, 1.0), (0.66996699669967,...

[1984 rows x 1 columns]>

In [37]:
df0.info

<bound method DataFrame.info of                                                  coords
0     [(0.75, 1.0), (0.5340909090909091, 0.946547884...
1     [(0.7154696132596685, 1.0), (0.511049723756906...
2     [(0.6941489361702128, 1.0), (0.486702127659574...
3     [(0.6851851851851852, 1.0), (0.473544973544973...
4     [(0.6487179487179487, 1.0), (0.446153846153846...
...                                                 ...
2650  [(0.5410447761194029, 1.0), (0.791044776119402...
2651  [(0.5243445692883895, 1.0), (0.767790262172284...
2652  [(0.5269230769230769, 1.0), (0.769230769230769...
2653  [(0.5225563909774437, 1.0), (0.766917293233082...
2654  [(0.5320754716981132, 1.0), (0.784905660377358...

[2655 rows x 1 columns]>

In [38]:
# The format of our csv has a list of data as a string, 
# to turn them into lists so we use the ast library
df1['coords'] = df1['coords'].apply(ast.literal_eval)
positive_data = np.array(df1['coords'].tolist())

df0['coords'] = df0['coords'].apply(ast.literal_eval)
negative_data = np.array(df0['coords'].tolist())

In [39]:
positive_data

array([[[0.66826923, 1.        ],
        [0.38461538, 0.93271462],
        [0.24038462, 0.78190255],
        ...,
        [0.97115385, 0.59860789],
        [0.85096154, 0.67981439],
        [0.77403846, 0.75638051]],

       [[0.66346154, 1.        ],
        [0.37980769, 0.93518519],
        [0.24038462, 0.78703704],
        ...,
        [0.97115385, 0.59953704],
        [0.85096154, 0.68287037],
        [0.77403846, 0.75925926]],

       [[0.66507177, 1.        ],
        [0.38277512, 0.93317972],
        [0.23923445, 0.78571429],
        ...,
        [0.98086124, 0.60368664],
        [0.86124402, 0.68663594],
        [0.77511962, 0.76267281]],

       ...,

       [[0.90939597, 1.        ],
        [0.66442953, 0.95943205],
        [0.5       , 0.81744422],
        ...,
        [0.99328859, 0.62677485],
        [0.94295302, 0.72413793],
        [0.90268456, 0.78701826]],

       [[0.90789474, 1.        ],
        [0.66118421, 0.96348884],
        [0.50328947, 0.82352941],
        .

In [40]:
negative_data

array([[[0.75      , 1.        ],
        [0.53409091, 0.94654788],
        [0.36079545, 0.82628062],
        ...,
        [0.95738636, 0.47216036],
        [0.98295455, 0.38975501],
        [1.        , 0.30957684]],

       [[0.71546961, 1.        ],
        [0.51104972, 0.94130435],
        [0.34530387, 0.8173913 ],
        ...,
        [0.95303867, 0.4826087 ],
        [0.98066298, 0.4       ],
        [1.        , 0.3173913 ]],

       [[0.69414894, 1.        ],
        [0.48670213, 0.93418259],
        [0.32446809, 0.80679406],
        ...,
        [0.94414894, 0.48619958],
        [0.97606383, 0.40127389],
        [1.        , 0.32059448]],

       ...,

       [[0.52692308, 1.        ],
        [0.76923077, 0.91666667],
        [0.90769231, 0.81321839],
        ...,
        [0.22692308, 0.50287356],
        [0.20384615, 0.40229885],
        [0.19230769, 0.32183908]],

       [[0.52255639, 1.        ],
        [0.76691729, 0.9154519 ],
        [0.91353383, 0.81924198],
        .

In [41]:
# We can concatenate them together to have all our data in one np array
data = np.concatenate((positive_data,negative_data))
data.shape

(4639, 21, 2)

In [42]:
# Each training example can be either 1 or 0
targets = np.zeros(data.shape[0])

# We are setting each training example to have either 0 or 1 output
for i in range(positive_data.shape[0]):
    targets[i] = 1

In [43]:
model = Sequential([
    Input(shape=(21,2)),          # Define the input shape here
    Flatten(),                     # Flatten the (21, 2) input into a 1D vector
    Dense(32, activation='relu'),  # Fully connected layer with 32 neurons
    Dense(16, activation='relu'),  # Fully connected layer with 16 neurons
    Dense(1, activation='sigmoid') # Output layer for binary classification
])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_3 (Flatten)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         1,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,921 (7.50 KB)

 Trainable params: 1,921 (7.50 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),  # Use appropriate loss function for classification
              metrics=['accuracy'])

In [45]:
history = model.fit(
    data,targets,
    epochs=40
)

Epoch 1/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 478us/step - accuracy: 0.8725 - loss: 0.5605
Epoch 2/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - accuracy: 0.9792 - loss: 0.1603
Epoch 3/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - accuracy: 0.9938 - loss: 0.0596
Epoch 4/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step - accuracy: 0.9980 - loss: 0.0288
Epoch 5/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - accuracy: 0.9974 - loss: 0.0162
Epoch 6/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - accuracy: 1.0000 - loss: 0.0106
Epoch 7/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step - accuracy: 0.9999 - loss: 0.0069
Epoch 8/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - accuracy: 1.0000 - loss: 0.0058
Epoch 9/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 1.0000 - loss: 0.0039
Epoch 10/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - accuracy: 1.0000 - loss: 0.0033
Epoch 11/40
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - accuracy: 1.0000 - loss: 0.0024
Epoch 12/40
145/145 ━━━━━━━━━━

In [46]:
import mediapipe as mp
import cv2

In [47]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1720225753.181978 15070763 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1720225753.211346 15097913 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1720225753.218399 15097913 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [48]:
cap = cv2.VideoCapture(0)
BOX_MARGIN = 50
while cap.isOpened():
    # capturing the current frame
    ret, frame = cap.read()
    # getting window dimensions, shape contains height, width, and channels
    height, width, _ = frame.shape

    # if no frame returned, break
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        # for each hand found we calculate the coords of the landmarks
        for hand_landmarks in results.multi_hand_landmarks:
            hand_coords = np.array([ (int(landmark.x * width), int(landmark.y * height)) for landmark in hand_landmarks.landmark ])
        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        hand_x_min, hand_x_max = min(x for x,y in hand_coords), max(x for x,y in hand_coords)
        hand_y_min, hand_y_max = min(y for x,y in hand_coords), max(y for x,y in hand_coords)
        
        rect_start, rect_end = (hand_x_min - BOX_MARGIN, hand_y_min - BOX_MARGIN), (hand_x_max + BOX_MARGIN, hand_y_max + BOX_MARGIN)
        rel_coords = np.array([ (x - rect_start[0], y - rect_start[1]) for x,y in hand_coords ])
        
        rel_x_max = max(x for x,y in rel_coords)
        rel_y_max = max(y for x,y in rel_coords)

        normalized_rel_coords = np.array([ (x / rel_x_max, y / rel_y_max) for x,y in rel_coords ])
        normalized_rel_coords = np.expand_dims(normalized_rel_coords, axis=0)

        prediction = model.predict(normalized_rel_coords)[0][0]
        if prediction > 0.9:
            text = f"peace be with you, Confidence: {round(prediction * 100,2)}%"
        else:
            text = f"give me some peace bro, Confidence: {round(prediction * 100,2)}%"

        cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
    cv2.imshow('Hand Landmarks', frame)

    # when q pressed, we end video capture
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


/opt/anaconda3/envs/tfsessions/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━

In [ ]:
type(prediction[0])